
# sklearn中的CV实现原理，best是怎么来的，score是怎么来的



search.fit(X,y)中的X和y全部用来训练。为了有效地选择超参数， X和y会被按照某种策略分成K份。针对每个超参数组合，进行K次训练。每次训练中K-1份挑出来作为train set用来训练，剩下的一份作为validation set用来测试计算（score）。K次计算得到的score按照某种策略进行平均作为这一组超参数的性能，选择最优超参数，重新训练模型得到best_estimator

最后使用新的X和y作为test set，测试模型的泛化能力

在每次训练中，使用skorch进行训练。K-1份数据进一步被分解成train set和validation set，实时监测训练进程，通过validation loss或者validation accuracy实现early stop 


# CV score不正确，原因需要显式调用StratifiedKFold

在构造RandomSearchCV的时候必须通过cv=StratifiedKFold(n_splits=3)显式调用StratifiedKFold，否则CV score计算不正确。这是因为直接使用cv=2，RandomSearchCV不会在每个类别中分别划分fold，而是在整个X上划分fold。这会导致训练集和测试集中数据分布不一致，X中数据没有进行shuffle，所以train set中仅包含排列在数组前面的类别，而validation set中仅包含排列在数组后面的类别。